In [12]:
import pandas as pd
import csv

# 读取原始 CSV 文件
df = pd.read_csv('ualberta_chinese_artifacts.csv')

# 字段对应的谓语
field_map = {
    'Accession Number': '编号',
    'Artist': '创作者',
    'Category': '类别',
    'Credit Line': '捐赠来源',
    'Culture': '文化',
    'Date Made': '制作时间',
    'Description': '描述',
    'Inscriptions and Colophons': '题跋',
    'Materials': '材质',
    'Materials Glossary': '材料术语',
    'Measurements': '尺寸',
    'Object Type': '对象类型',
    'Period': '时期',
    'Place Made': '制作地点',
    'Title': '标题'
}

# 用于指定每个谓语对应的目标节点标签
predicate_to_label = {
    '创作者': 'artist',
    '类别': 'category',
    '捐赠来源': 'donor',
    '文化': 'culture',
    '制作时间': 'date',
    '描述': 'description',
    '题跋': 'inscription',
    '材质': 'material',
    '材料术语': 'glossary',
    '尺寸': 'measurement',
    '对象类型': 'type',
    '时期': 'period',
    '制作地点': 'place',
    '标题': 'title'
}

triples = set()

for _, row in df.iterrows():
    subject = str(row.get('Accession Number', '未知编号')).strip()
    for field, predicate in field_map.items():
        if field == 'Accession Number':
            continue
        obj = row.get(field)
        if pd.notna(obj):
            obj_str = str(obj).strip().replace('\n', ' ')
            if obj_str.lower() in ['unknown', 'date unknown']:
                continue
            if obj_str:
                obj_label = predicate_to_label.get(predicate, 'Entity')
                triples.add((subject, predicate, obj_str, obj_label))

# 写入增强版 triples CSV
with open('triples_with_labels.csv', mode='w', newline='', encoding='utf-8-sig') as f:
    writer = csv.writer(f)
    writer.writerow(['Subject', 'Predicate', 'Object', 'ObjectLabel'])
    writer.writerows(triples)

print("✅ 已生成 triples_with_labels.csv，包含目标节点标签信息")


✅ 已生成 triples_with_labels.csv，包含目标节点标签信息


In [5]:
from neo4j import GraphDatabase

# 替换为你的 Neo4j Aura 连接信息
uri = "neo4j+s://e89c639b.databases.neo4j.io"
username = "neo4j"
password = "OJUkugsw_hiHzFkYO9Qq9NU-skLzv4E7CC-QfBAkkP0"  # 请替换为你 Aura 实例中提供的密码

# 创建驱动实例
driver = GraphDatabase.driver(uri, auth=(username, password))

# 测试查询函数
def test_connection():
    with driver.session() as session:
        result = session.run("MATCH (a)-[r]->(b) RETURN a.name AS from, type(r) AS rel, b.name AS to LIMIT 5")
        for record in result:
            print(f"{record['from']} -[{record['rel']}]-> {record['to']}")

# 执行测试
if __name__ == "__main__":
    try:
        test_connection()
        print("✅ 连接成功！")
    except Exception as e:
        print("❌ 连接失败：", e)
    finally:
        driver.close()


Album Leaf from Illustrations of Costumes from the Qianlong Court -[创作者]-> Huang Dao; Men Yingzhao; Jiang Renshu, and others
Album Leaf from Illustrations of Costumes from the Qianlong Court -[类别]-> albums
Album Leaf from Illustrations of Costumes from the Qianlong Court -[捐赠来源]-> Gift of Sandy and Cécile Mactaggart
Album Leaf from Illustrations of Costumes from the Qianlong Court -[制作时间]-> ca. 1759
Album Leaf from Illustrations of Costumes from the Qianlong Court -[描述]-> This album leaf is part of a section of the illustrations of costumes from the Qianlong court, Huangchao liqi tushi (“The Regulations for the Ceremonial Paraphernalia of the [Qing] Dynasty”). In total there are 34 folios that depict costumes appropriate for the Empress Dowager or Empress to wear at the imperial court along with short descriptive captions. This is the right leaf of one folio (attached to 2004.19.1.16) depicting the front of a formal court robe. The Chinese script describes the pictorial content of the 